In [ ]:
pip install flexvalue

In [ ]:
!flexvalue download-avoided-costs-data-db --year 2020

In [1]:
from google.colab import files
import pandas as pd

ModuleNotFoundError: No module named 'google'

In [2]:
from flexvalue.cet_flexvalue_compare import CET_Scan
from pathlib import Path

acc_version = '2020'

cet_scan = CET_Scan(
    directory='cet_scan',
    scan_name="Test_Run3",
    program_year="2021",
    acc_version=acc_version,
    program_admin="PGE",
    climate_zone=["12", "3A", "4"],
    mwh=[5000, 10, 2],
    therms=[200, 3, 114],
    units=[1, 1, 10],
    ntg=[0.95, 0.4, 0.9],
    eul=[7, 1, 1],
    sector=["NonRes", "Res", "Res"],
    deer_load_shape=[
        "DEER:HVAC_Chillers",
        "DEER:HVAC_Chillers",
        "DEER:Indoor_CFL_Ltg",
    ],
    gas_sector=["Residential", "Residential", "Residential"],
    gas_savings_profile=["Annual", "Winter Only", "Summer Only"],
    admin_cost=[5000, 165, 299],
    measure_cost=[220000, 310, 4044],
    incentive=[200000, 66, 800],
)

user_inputs = cet_scan.generate_cet_input_file()

Res/DEER:HVAC_Chillers Pairing Not Allowed in CET. Switching to Non_Res
Your CET input file is at cet_scan/Test_Run3/cet/Test_Run3.zip
Your FLEXvalue input file is at cet_scan/Test_Run3/flexvalue/Test_Run3_flexvalue_user_inputs.csv


In [ ]:
user_inputs

In [ ]:
from flexvalue import FlexValueRun
flx_run = FlexValueRun(database_year=acc_version)
outputs_table, outputs_table_totals, electric_benefits, gas_benefits = flx_run.get_results(user_inputs=user_inputs)

In [ ]:
from IPython.display import FileLink
# provide outputs table for download
outputs_filename = os.path.join(cet_scan.flexvalue_path, 'flexvalue_outputs.csv')
outputs_table.to_csv(outputs_filename)
display(FileLink(outputs_filename))
display(outputs_table)

In [ ]:
# Upload the cet output file
cet_output_dict = files.upload()
if len(metered_ls_dict) == 1:
    cet_output = list(cet_output_dict.values())[0]
else:
    print("Must select exactly one file")

In [ ]:
cet_cleaned_results = cet_scan.parse_cet_output()
compared_df = cet_scan.compare_cet_to_flexvalue(cet_cleaned_results, outputs_table)
compared_df

In [ ]:
from flexvalue.plots import plot_results
plot_results(outputs_table_totals, electric_benefits, gas_benefits)